In [1]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

print(tf.version)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


# Helper libraries
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import os.path
#import datetime, os

2023-03-11 23:06:34.448938: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 23:06:34.687740: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-11 23:06:35.860345: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/jarrad_foley135/miniconda3/envs/mlis_cw_car/lib/
2023-03-11 23:06:35.860449: W t

<module 'tensorflow._api.v2.version' from '/home/jarrad_foley135/miniconda3/envs/mlis_cw_car/lib/python3.9/site-packages/tensorflow/_api/v2/version/__init__.py'>
Num GPUs Available:  1


2023-03-11 23:06:37.501819: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-11 23:06:37.511135: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-11 23:06:37.511174: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
batch_size = 16
img_size = [240, 320]

df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
df['filename'] = df["image_id"].astype(str) + ".png"
#df['label'] = [df["angle"], df["speed"]]
print(df)


# x_train = df[0:int(len(df) * 0.6)]
#x_validate = df[int(len(df) * 0.6):int(len(df) * 0.8)]
x_evaluate = df[int(len(df) * 0.8):]

x_train = df[0:int(len(df) * 0.8)]
x_validate = df[int(len(df) * 0.8):]

training_images = tf.keras.preprocessing.image_dataset_from_directory(
    "machine-learning-in-science-ii-2023/training_data",
    image_size=img_size,
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1,
)


       image_id   angle  speed   filename
0             1  0.4375    0.0      1.png
1             2  0.8125    1.0      2.png
2             3  0.4375    1.0      3.png
3             4  0.6250    1.0      4.png
4             5  0.5000    0.0      5.png
...         ...     ...    ...        ...
13788     13794  0.6250    1.0  13794.png
13789     13795  0.4375    1.0  13795.png
13790     13796  0.5625    0.0  13796.png
13791     13797  0.6250    0.0  13797.png
13792     13798  0.6875    1.0  13798.png

[13793 rows x 4 columns]
Found 13798 files belonging to 1 classes.
Using 11039 files for training.


2023-03-11 23:06:38.227992: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 23:06:38.229618: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-11 23:06:38.229702: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-11 23:06:38.229737: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0

In [3]:
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest')

train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    batch_size=16,
    class_mode='other')

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    class_mode='other')

evaluate_datagen = ImageDataGenerator(rescale = 1./255)

eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    class_mode='other')


Found 11034 validated image filenames.
Found 2759 validated image filenames.
Found 2759 validated image filenames.


In [4]:
shape = (*img_size, 3) # inherited image size with 3 color filters

# CNN base
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=shape))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
        
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Dropout(0.2))
model.add(layers.Flatten()),

model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.7))

model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

model.add(layers.Dense(32, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.1))

# Output layer
model.add(layers.Flatten())
model.add(layers.Dense(2,  activation='relu', kernel_initializer='normal'))


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 238, 318, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 238, 318, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 119, 159, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 117, 157, 16)      4624      
                                                                 
 batch_normalization_1 (Batc  (None, 117, 157, 16)     64        
 hNormalization)                                                 
                                                        

In [6]:
model.compile(
    optimizer='adam',
    loss='mean_squared_error'
)

history = model.fit(
    train_generator, 
    batch_size=batch_size,
    validation_data=val_generator,
    epochs=5
)

Epoch 1/5


2023-03-11 23:06:43.261300: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-03-11 23:06:45.210089: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-11 23:06:49.113900: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied
2023-03-11 23:06:53.271922: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-11 23:06:53.298611: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x2dc7f1f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-11 23:06:53.298696: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVID

690/690 [==============================] - 62s 66ms/step - loss: 0.1255 - val_loss: 0.0430
Epoch 2/5
690/690 [==============================] - 45s 65ms/step - loss: 0.0467 - val_loss: 0.0281
Epoch 3/5
690/690 [==============================] - 47s 68ms/step - loss: 0.0390 - val_loss: 0.0263
Epoch 4/5
690/690 [==============================] - 41s 59ms/step - loss: 0.0336 - val_loss: 0.0336
Epoch 5/5
690/690 [==============================] - 39s 57ms/step - loss: 0.0335 - val_loss: 0.0254


In [7]:
#model = tf.keras.models.load_model('model.h6')

test_images = tf.keras.preprocessing.image_dataset_from_directory(
    "machine-learning-in-science-ii-2023/test_data",
    image_size=img_size,
    seed=1,
)

normalization_layer = tf.keras.layers.Rescaling(1./255)

test_images = test_images.map(lambda x, y: (normalization_layer(x), y))

import csv


prediction = model.predict(test_images)
print(prediction.shape)
import math

a= prediction[:,0]
b= prediction[:,1]
# b+=0.5
# b= np.floor(b)

submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('Submission.csv')


Found 1020 files belonging to 1 classes.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
32/32 [==============================] - 4s 104ms/step
(1020, 2)


In [8]:
from sklearn.metrics import classification_report

test_loss = model.evaluate(
    eval_generator,
    verbose=1
)

test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    target_size=img_size,
    class_mode=None)



#test_loss

predict_model = model.predict(test_generator, verbose =1)

# y = np.concatenate([y for x, y in x_evaluate], axis=0)
# true = np.argmax(y, axis = 1)

# print(true)
# predict_model = np.argmax(predict_model, axis = 1)
# print(predict_model)

# print(classification_report(true, predict_model, target_names=class_names))


87/87 [==============================] - 6s 70ms/step - loss: 0.0254
Found 2759 validated image filenames.
87/87 [==============================] - 6s 68ms/step
